# Question 1

We wish to train a machine learning algorithm on an array of floating point numbers in the interval [0.0, 1.0). The data is horribly unbalanced (not evenly distributed) and we wish to filter the dataset to obtain a subset containing an equal number of values from each interval [0, 0.2), [0.2, 0.4), ... [0.8, 1.0), throwing away as little data as possible.

1. Write a program which reads comma-separated floating point numbers in a single line from stdin, and prints the filtered data to stdout in the same format

**Overview**

How should we balance the data? If our goal is only to obtain a dataset with the same number of values in each bucket, then we can do several things

1. As the data comes in, we bucket the data, keeping track of how many elements are in each bucket. Then, at output time, we output that many elements from each bucket.
 - this does *throw away as little data as possible*

What does it do when there are no values in one interval [0.8, 1.0)?
* Return nothing for all the intervals?

Tests:

* Doesn't throw away data: if the number of elements in each bucket is the same to begin with



In [1]:
def cut(numbers, disp=False):
    """Filter the input to create a balanced data
    
    Params:
    numbers - array of floats in [0.0, 1.0)
    """
    intervals = [0.2, 0.4, 0.6, 0.8, 1.0]
    buckets = [[] for i in intervals]
    for num in numbers:
        # find the right bucket
        for i, right in enumerate(intervals):
            if num < right:
                buckets[i].append(num)
                break
    
    if disp:
        print('Buckets:\n', buckets)
        
    # find out how many values to keep
    k = min([len(b) for b in buckets])
    
    # take k values from each bucket
    buckets = [b[:k] for b in buckets]
    data = []
    for b in buckets:
        data += b
        
    return data


Let's test that `cut` works

In [2]:
# TEST: drops unbalanced elements
inputs = [0,0.1,0.3,0.4,0.5,0.7,0.9, 1.0]
expected = [0, 0.3, 0.4, 0.7, 0.9]
assert cut(inputs, True) == expected

# TEST: throws away as little as possible
inputs = [0, 0.1, 0.2, 0.3, 0.4, 0.4, 0.6, 0.6, 0.8, 0.8, 1.0, 1.0]
expected = [0, 0.1, 0.2, 0.3, 0.4, 0.4, 0.6, 0.6, 0.8, 0.8]
assert cut(inputs, True) == expected

Buckets:
 [[0, 0.1], [0.3], [0.4, 0.5], [0.7], [0.9]]
Buckets:
 [[0, 0.1], [0.2, 0.3], [0.4, 0.4], [0.6, 0.6], [0.8, 0.8]]


In [3]:
!echo 0, 0.1, 0.2, 0.3, 0.4, 0.4, 0.6, 0.6, 0.8, 0.8, 1.0, 1.0 | python filter.py


0.0,0.1,0.2,0.3,0.4,0.4,0.6,0.6,0.8,0.8


Of course, we'll need some end-to-end tests to make sure that the program output is what we expect

In [4]:
# TODO

# Question 4

Write a program which counts how many possible games of TicTacToe consist of a certain number of moves.

Output format example (these numbers are incorrect):
```
5 78492
6 289
7 48901
8 8172
9 231
```

How do we do this efficiently? 

In [5]:
# basic representation

# the board 
# [ 0 1 2 ]
# [ 3 4 5 ]
# [ 6 7 8 ]

wins = [
    # horizontal
    [0,1,2],
    [3,4,5],
    [6,7,8],
    
    # vertical
    [0,3,6],
    [1,4,7],
    [2,5,8],
    
    # diagonal
    [0,4,8],
    [2,4,6]
]

# the idea is to only check the wins that occur for each square
# on second thought, that didn't save us that much time
possible_wins = {
    square:[w for w in wins if square in w] for square in range(9)
}

possible_wins

{0: [[0, 1, 2], [0, 3, 6], [0, 4, 8]],
 1: [[0, 1, 2], [1, 4, 7]],
 2: [[0, 1, 2], [2, 5, 8], [2, 4, 6]],
 3: [[3, 4, 5], [0, 3, 6]],
 4: [[3, 4, 5], [1, 4, 7], [0, 4, 8], [2, 4, 6]],
 5: [[3, 4, 5], [2, 5, 8]],
 6: [[6, 7, 8], [0, 3, 6], [2, 4, 6]],
 7: [[6, 7, 8], [1, 4, 7]],
 8: [[6, 7, 8], [2, 5, 8], [0, 4, 8]]}

In [6]:
def checkWin(board, pattern):
    return board[pattern[0]] == board[pattern[1]] and board[pattern[0]] == board[pattern[2]]

def pprint(board):
    print(board[:3])
    print(board[3:6])
    print(board[6:])
    print('--------------')
    
def calculateWinNumber(game, disp=False):
    board = [' '] * 9
    mark = 'x'
    for moveNumber, move in enumerate(game):
        move = int(move)
        board[move] = mark
        if disp:
            pprint(board)
        mark = 'o' if mark == 'x' else 'x'
        # check for win
        for pattern in possible_wins[move]:
            if checkWin(board, pattern):
                return moveNumber + 1
    
    # tie? / no one wins
    return -1

print(calculateWinNumber('40236', True))

[' ', ' ', ' ']
[' ', 'x', ' ']
[' ', ' ', ' ']
--------------
['o', ' ', ' ']
[' ', 'x', ' ']
[' ', ' ', ' ']
--------------
['o', ' ', 'x']
[' ', 'x', ' ']
[' ', ' ', ' ']
--------------
['o', ' ', 'x']
['o', 'x', ' ']
[' ', ' ', ' ']
--------------
['o', ' ', 'x']
['o', 'x', ' ']
['x', ' ', ' ']
--------------
5


In [8]:
import itertools
import random

def bruteForce(LOG=False, SAMPLE=False):
    # a game is represented by 0-8
    # ex: 012345678
    
    counters = [0 for i in range(12)] # the last one is -1

    all_permutations = itertools.permutations('012345678')
    counter = 0
    for game in all_permutations:
        
        # log
        counter += 1
        if LOG and counter % 1000 == 0:
            print(counter // 1000, counters)
        
        # sample games to see if it's calculating correctly
        if SAMPLE and random.random() < 0.001:
            print(calculateWinNumber(game, True))
            
        counters[calculateWinNumber(game)] += 1
    
    # format it in the right format
    for i in [5,6,7,8,9]:
        k = counters[i]
        print("{} {}".format(i, k))
        
    return counters
        

bruteForce()

5 34560
6 31968
7 95904
8 72576
9 81792


[0, 0, 0, 0, 0, 34560, 31968, 95904, 72576, 81792, 0, 46080]

# Question 5

In [10]:
FORWARD = 'f'
BACK = 'b'
SKIP = 's'
JUMP = list(range(10))

def crater(x):
    return x == 10

class Interpreter:
    """
    We're using a basic interpreter loop here...
    
    """
    def execute(self, program, initial_pos = 10, LIMIT=30):
        position = initial_pos
        
        counter = 0
        time = 0
        skip = False
        while counter < len(program):
            time += 1
            if time > LIMIT:
                print("CAR: ran out of gas, sorry")
                break
            
            if skip:
                skip = False
                if crater(position):
                    print('Encountered crater!')
                    counter += 1
                    continue
            
            instruction = program[counter]
            print(counter, instruction, position)
            
            if instruction == FORWARD:
                position += 1
            elif instruction == BACK:
                position -= 1
            elif instruction == SKIP:
                skip = True
            elif instruction in JUMP:
                counter = instruction
                continue
            
            counter += 1
                
c = Interpreter()
c.execute([
    FORWARD, BACK, FORWARD, BACK, SKIP, FORWARD, BACK, SKIP
])

0 f 10
1 b 11
2 f 10
3 b 11
4 s 10
Encountered crater!
6 b 10
7 s 9


In [11]:
# TODO: finish the program for both cars!